# Dhruv Rana #

In [25]:
import gym
from gym.wrappers.monitoring.video_recorder import VideoRecorder

In [26]:
env = gym.make("LunarLander-v2",render_mode='rgb_array')
Video = VideoRecorder(env,"launch.mp4")

done = False
i = 0

env.reset()
while (not done) or (i <= 100):
    env.render()
    Video.capture_frame()
    
    action = env.action_space.sample()
    
    observation,reward,done,info,_ = env.step(action)
    print("action for step",i,action)
    print("step",i,observation,reward,done,info)
    i = i + 1
    
Video.close()
env.close()

from IPython.display import HTML

HTML("""
<video controls>
    <source src = "%s" type ="video/mp4">
</video>
"""%"launch.mp4"
)

Moviepy - Building video launch.mp4.
Moviepy - Writing video launch.mp4



Moviepy - Done !
Moviepy - video ready launch.mp4
action for step 0 2
step 0 [-0.01454267  1.4175118  -0.7413524   0.15554264  0.01592355  0.15135604
  0.          0.        ] -2.8924340569863487 False False
action for step 1 0
step 1 [-0.02187986  1.4204141  -0.7413759   0.12888488  0.02348635  0.15127003
  0.          0.        ] -0.5560698935177015 False False
action for step 2 0
step 2 [-0.02921734  1.4227171  -0.74139833  0.10221386  0.03104794  0.1512461
  0.          0.        ] -0.5911570082123774 False False
action for step 3 0
step 3 [-0.03655519  1.4244208  -0.7414207   0.07554173  0.03860833  0.15122208
  0.          0.        ] -0.6281202531961299 False False
action for step 4 3
step 4 [-0.04381914  1.4255307  -0.732159    0.04916999  0.04430285  0.11390094
  0.          0.        ] 0.414215978119189 False False
action for step 5 3
step 5 [-0.05099259  1.4260583  -0.72077096  0.02333835  0.04769866  0.06792257
  0.          0.        ] 0.8198066354300988 False False
action

Moviepy - Done !
Moviepy - video ready launch.mp4
